# Grid Search of parameters

In [1]:
from wordle_base import run_simulations as rl_base
from wordle_cluster import run_simulations as rl_cluster
from wordle_cluster_2 import run_simulations as rl_cluster_2
import pandas as pd

learning_rates = [0.1, 0.01]    
exploration_rates = [0.1, 0.5, 0.6, 0.7, 0.8, 0.9]
shrinkage_factors = [0.5, 0.6, 0.7, 0.8, 0.9]
num_of_clusters = [1, 2, 3, 4, 5]

In [2]:
df_1 = pd.DataFrame(columns=['learning_rate', 'exploration_rate', 'shrinkage_factor', 'time_taken', 'average_guesses', 'win_rate'])

for i, alpha in enumerate(learning_rates):
    for j, epsilon in enumerate(exploration_rates):
        for k, gamma in enumerate(shrinkage_factors):
            print(f'Running epoch {i}, {j}, {k}')
            time_taken, average_guesses, win_rate = rl_base(learning_rate=alpha, exploration_rate=epsilon, shrinkage_factor=gamma, num_simulations=100)
            df_1.loc[len(df_1)] = [alpha, epsilon, gamma, time_taken, average_guesses, win_rate]
            
df_1.to_csv('results_base.csv', index=False)
df_1.loc[df_1['win_rate'].idxmax()]

KeyboardInterrupt: 

In [ ]:
df_2 = pd.DataFrame(columns=['learning_rate', 'exploration_rate', 'shrinkage_factor', 'num_of_clusters', 'time_taken', 'average_guesses', 'win_rate'])

for alpha in learning_rates:
    for epsilon in exploration_rates:
        for gamma in shrinkage_factors:
            for num_clusters in num_of_clusters:
                print(f'Running epoch {i}, {j}, {k}')
                time_taken, average_guesses, win_rate = rl_cluster(learning_rate=alpha, exploration_rate=epsilon, shrinkage_factor=gamma, number_of_cluster=num_clusters, num_simulations=100)
                df_2.loc[len(df_2)] = [alpha, epsilon, gamma, num_clusters, time_taken, average_guesses, win_rate]

df_2.to_csv('results_cluster.csv', index=False)
df_2.loc[df_2['win_rate'].idxmax()]

In [ ]:
df_3 = pd.DataFrame(columns=['learning_rate', 'exploration_rate', 'shrinkage_factor', 'num_of_clusters', 'time_taken', 'average_guesses', 'win_rate'])

for alpha in learning_rates:
    for epsilon in exploration_rates:
        for gamma in shrinkage_factors:
            for num_clusters in num_of_clusters:
                print(f'Running epoch {i}, {j}, {k}')
                time_taken, average_guesses, win_rate = rl_cluster_2(learning_rate=alpha, exploration_rate=epsilon, shrinkage_factor=gamma, number_of_cluster=num_clusters, num_simulations=100)
                df_3.loc[len(df_3)] = [alpha, epsilon, gamma, num_clusters, time_taken, average_guesses, win_rate]

df_3.to_csv('results_cluster_2.csv', index=False)
df_3.loc[df_3['win_rate'].idxmax()]